In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("adult.data.csv")
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
data["sex"].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

In [4]:
data_male = data[data["sex"] == "Male"]
data_male["age"].mean()

39.43354749885268

In [5]:
data['native-country'].value_counts().loc['United-States'] / len(data) * 100

89.5857006848684

In [6]:
def func():
    dm = data.groupby('salary')[['age']].mean()
    ds = data.groupby('salary')[['age']].std()
 
    d = pd.concat([dm,ds],axis=1)
    d.columns = ['mean','std']
    print(d)
func()

             mean        std
salary                      
<=50K   36.783738  14.020088
>50K    44.249841  10.519028


In [7]:
data[data['salary'] == '>50K']['education'].unique()

array(['HS-grad', 'Masters', 'Bachelors', 'Some-college', 'Assoc-voc',
       'Doctorate', 'Prof-school', 'Assoc-acdm', '7th-8th', '12th',
       '10th', '11th', '9th', '5th-6th', '1st-4th'], dtype=object)

In [8]:
age_describe = data.groupby(['race', 'sex'])['age'].describe()

print(f'Maximum age for Asian-Pac-Islander males - {age_describe["max"]["Asian-Pac-Islander"]["Male"]}')
age_describe

Maximum age for Asian-Pac-Islander males - 90.0


count       mean        std   min   25%   50%  \
race               sex                                                       
Amer-Indian-Eskimo Female    119.0  37.117647  13.114991  17.0  27.0  36.0   
                   Male      192.0  37.208333  12.049563  17.0  28.0  35.0   
Asian-Pac-Islander Female    346.0  35.089595  12.300845  17.0  25.0  33.0   
                   Male      693.0  39.073593  12.883944  18.0  29.0  37.0   
Black              Female   1555.0  37.854019  12.637197  17.0  28.0  37.0   
                   Male     1569.0  37.682600  12.882612  17.0  27.0  36.0   
Other              Female    109.0  31.678899  11.631599  17.0  23.0  29.0   
                   Male      162.0  34.654321  11.355531  17.0  26.0  32.0   
White              Female   8642.0  36.811618  14.329093  17.0  25.0  35.0   
                   Male    19174.0  39.652498  13.436029  17.0  29.0  38.0   

                             75%   max  
race               sex                  
Amer-Indian-Eskimo Female  46.00  80.0  
                   Male    45.00  82.0  
Asian-Pac-Islander Female  43.75  75.0  
                   Male    46.00  90.0  
Black              Female  46.00  90.0  
                   Male    46.00  90.0  
Other              Female  39.00  74.0  
                   Male    42.00  77.0  
White              Female  46.00  90.0  
                   Male    49.00  90.0

In [9]:
status = [ i for i in data['marital-status'].unique() if i.startswith('Married') ]

married = data[(data['sex'] == 'Male') &
     (data['marital-status'].isin(status))]['salary'].value_counts()
print(married)
not_married = data[(data['sex'] == 'Male') &
     (~data['marital-status'].isin(status))]['salary'].value_counts()
print(not_married)
print("Женаты: {0}%".format(round(100 * married[1]/(married[0]+married[1]), 2)))
print("Холосты: {0}%".format(round(100 * not_married[1]/(not_married[0]+not_married[1]), 2)))

<=50K    7576
>50K     5965
Name: salary, dtype: int64
<=50K    7552
>50K      697
Name: salary, dtype: int64
Женаты: 44.05%
Холосты: 8.45%


In [10]:
max_work = data['hours-per-week'].max()
print("Максимальное число часов в неделю - {0} часов".format(max_work))

num_workers = data[data['hours-per-week'] == max_work].shape[0]
print("Всего людей с таким количеством часов - {0}".format(num_workers))

per_rich = data[(data['hours-per-week'] == max_work) & (data['salary'] == '>50K')].shape[0] / num_workers
print("Из них богатых - {0}%".format(round(100 * per_rich, 2)))

Максимальное число часов в неделю - 99 часов
Всего людей с таким количеством часов - 85
Из них богатых - 29.41%


In [11]:
avg_work = data.groupby([data['native-country'], data['salary']])['hours-per-week'].mean()
avg_work

native-country  salary
?               <=50K     40.164760
                >50K      45.547945
Cambodia        <=50K     41.416667
                >50K      40.000000
Canada          <=50K     37.914634
                            ...    
United-States   >50K      45.505369
Vietnam         <=50K     37.193548
                >50K      39.200000
Yugoslavia      <=50K     41.600000
                >50K      49.500000
Name: hours-per-week, Length: 82, dtype: float64

In [12]:
pd.cut(data['age'], bins=3).value_counts()
cut_labels_3 = ['joung', 'adult', 'retiree']
cut_bins = [16, 35, 70, 100]

data['AgeGroup'] = pd.cut(data['age'],
                       bins=cut_bins,
                       labels=cut_labels_3)
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,AgeGroup
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,adult
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,adult
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,adult
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,adult
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,joung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,joung
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,adult
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,adult
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,joung


In [13]:
more_50K = data.where(data['salary'] == '>50K').groupby('AgeGroup')['salary'].count()

print(f'earns over 50K - {more_50K.idxmax()}')
more_50K

earns over 50K - adult


AgeGroup
joung      1705
adult      6042
retiree      94
Name: salary, dtype: int64

In [15]:
occupation = data.groupby("occupation").size()
print(occupation)

occupation
?                    1843
Adm-clerical         3770
Armed-Forces            9
Craft-repair         4099
Exec-managerial      4066
Farming-fishing       994
Handlers-cleaners    1370
Machine-op-inspct    2002
Other-service        3295
Priv-house-serv       149
Prof-specialty       4140
Protective-serv       649
Sales                3650
Tech-support          928
Transport-moving     1597
dtype: int64


In [16]:
def filter_func(df):
      df = df.groupby("occupation").filter(lambda x: (x["age"].mean() < 40) & (x["hours-per-week"].min() > 5))
      return df["occupation"].unique()

filter_func(data)

array(['Armed-Forces'], dtype=object)